In [25]:
import pandas as pd
from matplotlib import pyplot as plt
import nltk as nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from _keys import db_user, db_password, db_name, db_host, db_port
import psycopg2

In [26]:
nltk.download('stopwords')
stop=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\steve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
sql = """
select all_events.subreddit, all_events.submission, all_events.submission_type
from
	(select *
	from (
	select author, title as submission, created_utm as time_created, subreddit, 'post' as submission_type
	from sus_user_posts
	where title !='This account is banned and is temporarily preserved for purposes of transparency.') as posts_aggregate
	union
	select *
	from (
	select author, comment_body as submission, created_utc as time_created, subreddit, 'comment' as submission_type
	from sus_user_comments) as comments_aggregate) as all_events
where length(submission) > 0 and subreddit not in ('funny', 'aww') and submission_type = 'post'
"""
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)

In [28]:
df = pd.read_sql(sql, conn)

In [31]:
def generate_top_ngrams(text, n=3):
    stop=set(stopwords.words('english'))

    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]

    def _get_top_ngram(corpus, n=None):
        vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) 
                      for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:10]

    top_n_bigrams=_get_top_ngram(text,n)[:50]
    x,y=map(list,zip(*top_n_bigrams))
    return list(zip(x,y))

In [32]:
nlp_df = pd.DataFrame(generate_top_ngrams(df['submission']), columns=['ngram','count'])
nlp_df.head(15)

,ngram,count
0,black lives matter,34
1,year old boy,24
2,what do you,23
3,in the us,21
4,do you think,20
5,year old girl,19
6,in front of,19
7,shot by police,18
8,12 year old,17
9,one of the,16
